# Comparing Community Detection Methods on the Same Student Graph

**Goal :** compare several community detection algorithms on the **same k-NN student graph**:
- Louvain 
- Greedy Modularity 
- Girvan–Newman
- Infomap 

### Part 1 : import et config 

In [33]:
import sys, os, importlib
import pandas as pd
import networkx as nx
import numpy as np
from collections import Counter, defaultdict
from sklearn.metrics import adjusted_rand_score as ARI, normalized_mutual_info_score as NMI

sys.path.insert(0, os.path.abspath('..'))
import src.data_preprocessing as dp
import src.graph_students as gs
importlib.reload(dp); importlib.reload(gs)

DATA_DIR = "../data"
MODULE = "AAA"
PRESENTATION = "2013J"

# Graph parameters (we use the best paramater found)
K = 10
TAU = 0.20

### Part 2 : load data + construction of the same kNN graph 

In [34]:
student_data, X_raw, assess = dp.load_and_prepare_oulad(DATA_DIR, MODULE, PRESENTATION)

# similarity + k-NN graph
X = gs.normalize_rows_max(X_raw) if hasattr(gs, "normalize_rows_max") else X_raw
sim = gs.build_similarity_matrix(X)
G = gs.build_knn_graph(sim, student_ids=X.index.tolist(), k=K, sim_threshold=TAU)

print(f"|V|={G.number_of_nodes()}, |E|={G.number_of_edges()}, density={nx.density(G):.4f}")

|V|=378, |E|=3053, density=0.0428


### Part 3 : Structural comparison 

In [35]:
methods_to_try = ["louvain", "greedy", "girvan_newman", "infomap"]
rows = []
parts = {}

for m in methods_to_try:
    try:
        part = gs.detect_communities(G, method=m)
        parts[m] = part
        ev = gs.evaluate_partition(G, part)
        rows.append({
            "method": m,
            "modularity": round(ev["modularity"], 3),
            "n_communities": ev["n_communities"],
            "largest_communities": sorted(ev["sizes"].values(), reverse=True)[:5]
        })
    except Exception as e:
        rows.append({
            "method": m,
            "modularity": None,
            "n_communities": None,
            "largest_communities": None,
            "error": str(e)
        })

df_compare = pd.DataFrame(rows)
df_compare

,method,modularity,n_communities,largest_communities
0,louvain,0.531,7,"[101, 70, 69, 64, 34]"
1,greedy,0.481,3,"[140, 124, 114]"
2,girvan_newman,0.334,5,"[268, 80, 15, 14, 1]"
3,infomap,0.521,12,"[99, 72, 44, 30, 26]"


#### Comparison of community detection methods

We run all methods on the **same k-NN graph** (*k = 10, τ = 0.20*).

| Method | Modularity | Comms | Largest group share | Avg size (= N / #comms) | Comment |
|-------|-----------:|:-------:|--------------------:|-------------------------:|---------|
| **Louvain** | **0.531** | 7 | **26.7%** (101/378) | ~54 | High Q and **balanced** groups → good structure. |
| **Greedy** | 0.481 | 3 | **37.0%** (140/378) | ~126 | Few big clusters → simple to read, less detail. |
| **Girvan–Newman** | 0.334 | 5 | **70.9%** (268/378) | ~76 | One dominant cluster → weak partition for this data. |
| **Infomap** | **0.526** | 11 | **25.4%** (96/378) | ~34 | High Q but **fine granularity** (many small groups). |

**What these numbers say**

- **Strength of structure (modularity):** Louvain and Infomap (~0.53) detect the strongest community structure.  
- **Granularity:** Greedy = coarse (3 groups), Louvain = medium (7), Infomap = fine (11).  
- **Balance of sizes:** Louvain keeps groups **balanced** (largest only 26.7%).  
  Girvan–Newman is **unbalanced** (one group = 70.9% of students), so the split is not very meaningful here.

**Takeaways**

- Use **Louvain** when you want a **good balance** between quality (high Q) and readability (medium #groups).  
- Use **Greedy** for a quick, **simple** picture (3 big clusters).  
- Use **Infomap** when you need **sub-communities** and more detail.  
- Avoid **Girvan–Newman** for this dataset (low Q, one dominant cluster, slow).


### Part 4 : Agreement between ARI / NMI method 

In [24]:
def labels_from_part(part, nodes):
    return [part[n] for n in nodes]

nodes = list(G.nodes())
pairs = []
for a in ["louvain","greedy","infomap"]:
    if a not in parts: continue
    for b in ["louvain","greedy","infomap"]:
        if b not in parts or a >= b: continue
        la, lb = labels_from_part(parts[a], nodes), labels_from_part(parts[b], nodes)
        pairs.append({"pair": f"{a} vs {b}",
                      "ARI": round(ARI(la, lb), 3),
                      "NMI": round(NMI(la, lb), 3)})
pd.DataFrame(pairs)

,pair,ARI,NMI
0,greedy vs louvain,0.501,0.591
1,greedy vs infomap,0.437,0.548
2,infomap vs louvain,0.731,0.773


#### Quantitative comparison between community detection methods

We compared the three main methods — **Greedy**, **Louvain**, and **Infomap** —  
using two additional metrics:

| Metric | Meaning |
|---------|----------|
| **ARI (Adjusted Rand Index)** | How similar two partitions are (1 = identical, 0 = very different). |
| **NMI (Normalized Mutual Information)** | Measures agreement in information content between partitions. |
| **Purity (w.r.t final_result)** | How homogeneous each community is regarding students’ academic outcomes. |

---

#### Agreement between methods

| Pair | ARI | NMI | Interpretation |
|------|-----|-----|----------------|
| Greedy ↔ Louvain | **0.50** | **0.59** | Medium agreement: both find similar global groups. |
| Greedy ↔ Infomap | **0.44** | **0.55** | Moderate overlap: Infomap splits Greedy’s groups into smaller ones. |
| Infomap ↔ Louvain | **0.73** | **0.77** | Strong agreement: both describe almost the same community structure. |

**Interpretation:**  
- Louvain and Infomap are highly consistent (they detect almost the same structure).  
- Greedy captures the same main clusters, but in a more aggregated (coarse) way.  
- This confirms that the **Louvain graph** is a good balance between structure and interpretability.


### Part 5 : link with the reussite

In [37]:
def partition_purity(part: dict, meta_df: pd.DataFrame, label_col="final_result"):
    # 1) garder uniquement les étudiants présents dans la partition
    idx = meta_df.index.intersection(part.keys())
    df = meta_df.loc[idx].copy()
    df["community"] = pd.Series(part)

    # 2) ignorer les lignes où le label est manquant
    df = df.dropna(subset=[label_col, "community"])

    # 3) calcul pondéré par la taille de chaque communauté
    groups = df.groupby("community")
    tot = sum(len(g) for _, g in groups)  # pas len(df) si des NaN avaient été filtrés
    purity = 0.0
    for _, grp in groups:
        if grp.empty: 
            continue
        m = grp[label_col].value_counts(normalize=True).max()
        purity += m * (len(grp) / tot)
    return purity

meta = student_data.set_index("id_student").loc[X.index]

rows_more = []
for m in ["louvain", "greedy", "infomap"]:
    if m in parts:
        pur = partition_purity(parts[m], meta, label_col="final_result")
        rows_more.append({"method": m, "purity_final_result": round(pur, 3)})

df_compare = df_compare.merge(pd.DataFrame(rows_more), on="method", how="left")
df_compare

,method,modularity,n_communities,largest_communities,purity_final_result
0,louvain,0.531,7,"[101, 70, 69, 64, 34]",0.697
1,greedy,0.481,3,"[140, 124, 114]",0.668
2,girvan_newman,0.334,5,"[268, 80, 15, 14, 1]",NaN
3,infomap,0.521,12,"[99, 72, 44, 30, 26]",0.700



##### Relation with students’ outcomes (purity)

| Method | Purity (final_result) | Comment |
|---------|----------------------:|----------|
| **Louvain** | **0.697** | Balanced and coherent communities; very good link with success rates. |
| **Greedy** | 0.668 | Fewer, larger groups — less homogeneous but easier to interpret |
| **Infomap** | **0.700** | Slightly higher purity: detects finer and highly consistent subgroups |

**Interpretation:**  
- **Infomap** gives the highest purity → its small communities are strongly aligned with students’ results.  
- **Louvain** performs almost as well as infomap.  
- **Greedy** produces broader clusters that remain meaningful but mix different student profiles.
- **Girvan–Newman** fails to identify coherent learning groups in this context.
